In [2]:
import pandas as pd
import utils
import plotly.graph_objects as go
import instruments


ModuleNotFoundError: No module named 'utils'

In [ ]:
symbol ="SPX500_USD"
granularity = "H1"
ma_list = [5,13]
i_pair = instruments.Instrument.GetInstrumentSymbol(symbol)
print(i_pair)

{'name': 'SPX500_USD', 'ins_type': 'CFD', 'displayName': 'US SPX 500', 'pipLocation': 1, 'marginRate': 0.05}


In [49]:
df = pd.read_csv(utils.get_his_data_filename(symbol ,granularity))
non_cols = ["time","volume"]
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)


In [50]:
df_ma= df[["time","mid_o","mid_h","mid_l","mid_c"]].copy()
df_plot = df_ma
for ma in ma_list:
    df_ma[f"MA_{ma}"] = df_ma["mid_c"].rolling(window=ma).mean()
df_ma.dropna(inplace=True)  
df_ma.reset_index(inplace=True, drop=True)  


In [51]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_5,MA_13
0,2021-03-05T18:00:00.000000000Z,3795.4,3822.8,3783.0,3817.6,3782.20,3773.530769
1,2021-03-05T19:00:00.000000000Z,3817.8,3836.6,3803.4,3835.0,3793.72,3778.807692
2,2021-03-05T20:00:00.000000000Z,3835.4,3852.0,3829.2,3840.4,3810.24,3784.992308
3,2021-03-05T21:00:00.000000000Z,3840.2,3845.0,3838.1,3842.8,3826.24,3791.661538
4,2021-03-07T23:00:00.000000000Z,3856.2,3868.1,3853.0,3856.2,3838.40,3799.461538


In [52]:
def TradeSignal(row):
    """
    This function returns trade signal FLAG in DataFrame
    """
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1 
    if row.DIFF <= 0 and row.DIFF_PREV >0:
        return -1
    return 0

In [53]:
df_ma["DIFF"] = df_ma.MA_5 - df_ma.MA_13 # Get the MA difference for crossover signal
df_ma["DIFF_PREV"] = df_ma.DIFF.shift(1) # 
df_ma["TRADE_SIGNAL"] = df_ma.apply(TradeSignal, axis=1) # Apply function TradeSignal to DataFrame column
df_trades = df_ma[df_ma.TRADE_SIGNAL!=0].copy()
df_trades["DELTA"] = ( df_trades.mid_c.diff() / i_pair.pipLocation).shift(-1)
df_trades["GAIN"] = df_trades["DELTA"]*df_trades["TRADE_SIGNAL"]
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_5,MA_13,DIFF,DIFF_PREV,TRADE_SIGNAL,DELTA,GAIN
11,2021-03-08T06:00:00.000000000Z,3830.6,3836.7,3826.4,3828.0,3831.68,3834.638462,-2.958462,6.776923,-1,15.8,-15.8
19,2021-03-08T14:00:00.000000000Z,3844.0,3864.2,3831.8,3843.8,3828.96,3826.930769,2.029231,-5.270769,1,-3.9,-3.9
27,2021-03-08T23:00:00.000000000Z,3832.9,3844.1,3832.1,3839.9,3838.52,3841.215385,-2.695385,5.921538,-1,9.7,-9.7
33,2021-03-09T05:00:00.000000000Z,3852.6,3855.3,3849.1,3849.6,3845.76,3845.130769,0.629231,-3.455385,1,23.6,23.6
51,2021-03-10T00:00:00.000000000Z,3877.5,3877.9,3870.6,3873.2,3878.78,3878.823077,-0.043077,6.775385,-1,9.0,-9.0


In [54]:
df_trades.GAIN.sum()

676.199999999998

In [81]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open = df_plot.mid_o, high = df_plot.mid_h, low = df_plot.mid_l, close = df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor="#0069d9",
    increasing_line_color="#659fd8"
))
for ma in ma_list:
    col=f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time,
        y=df_plot[col],
        line=dict(width=2),
        line_shape="spline",
        name=col))
fig.update_layout(width=1200,height=700,
    margin= dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor ="#1e1e1e",
    plot_bgcolor ="#1e1e1e",)
fig.update_xaxes(
    gridcolor ="#1f232f",
    showgrid=True
)
fig.update_yaxes(
    gridcolor ="#1f232f",
    showgrid=True
)

fig.show()